In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms, models
from diffusers import DDPMPipeline
import numpy as np
import random

In [8]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

In [9]:
class DDPMWithFeatures(nn.Module):
    def __init__(self, pipeline):
        super(DDPMWithFeatures, self).__init__()
        self.pipeline = pipeline
        self.unet = pipeline.unet

    def forward(self, sample, timestep, layer_indices):
        x = sample
        t_emb = self.unet.time_proj(timestep)
        t_emb = self.unet.time_embedding(t_emb)

        # Initial convolution
        x = self.unet.conv_in(x)

        features = {}
        # Downsampling blocks
        for i, down_block in enumerate(self.unet.down_blocks):
            x_tuple = down_block(x, t_emb)
            x, _ = x_tuple

            if i in layer_indices:
                features[i] = x

        # Middle block
        mid_output = self.unet.mid_block(x, t_emb)
        if isinstance(mid_output, tuple):
            x = mid_output[0]
        else:
            x = mid_output

        if len(self.unet.down_blocks) in layer_indices:
            features[len(self.unet.down_blocks)] = x

        return features


In [10]:
def extract_features_by_layer(data_loader, model, layer_indices, device):

    all_features = {layer: [] for layer in layer_indices}
    all_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            batch_size = images.size(0)

            # Use t=0 for clean images
            timesteps = torch.tensor([0] * batch_size, device=device).long()

            # Extract features from multiple layers of the diffusion model
            features = model(images, timesteps, layer_indices)

            for layer_index in layer_indices:
                # Get the feature tensor from the specified layer
                layer_features = features[layer_index]
                layer_features = layer_features.reshape(batch_size, -1)

                # Store the features for this layer
                all_features[layer_index].append(layer_features.cpu())

            all_labels.append(labels)

    # Concatenate features and labels for each layer
    for layer_index in layer_indices:
        all_features[layer_index] = torch.cat(all_features[layer_index])
    all_labels = torch.cat(all_labels)

    return all_features, all_labels


In [11]:
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, num_classes,dropout=0.5):
        super(MLPClassifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)

In [12]:
def train_classifier(features, labels, input_dim, device, num_classes=10, epochs=10, batch_size=128, learning_rate=0.001,dropout=0.5):

    # Define the classifier model (simple MLP with one hidden layer and ReLU activation)
    model = MLPClassifier(input_dim=input_dim, num_classes=num_classes,dropout=dropout).to(device)

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Create a DataLoader for training
    dataset = torch.utils.data.TensorDataset(features, labels)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Training loop
    model.train()  # Set model to training mode
    for epoch in range(epochs):
        running_loss = 0.0

        # Loop over batches
        for inputs, lbls in loader:
            inputs, lbls = inputs.to(device), lbls.to(device)  # Move data to the specified device

            optimizer.zero_grad()  # Reset gradients

            outputs = model(inputs)  # Forward pass through the model

            loss = criterion(outputs, lbls)  # Compute loss
            loss.backward()  # Backward pass to compute gradients

            optimizer.step()  # Update model parameters

            running_loss += loss.item() * inputs.size(0)  # Accumulate loss

        # Calculate average loss for the epoch
        epoch_loss = running_loss / len(dataset)
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}')

    return model

In [13]:
def evaluate_classifier(model, test_features, test_labels, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i in range(0, len(test_features), 128):
            inputs = test_features[i:i+128].to(device)
            lbls = test_labels[i:i+128].to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += lbls.size(0)
            correct += (predicted == lbls).sum().item()

    accuracy = 100 * correct / total
    return accuracy


In [21]:
# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the pre-trained DDPM model from Hugging Face
model_name = "google/ddpm-cifar10-32"
ddpm_pipeline = DDPMPipeline.from_pretrained(model_name)
ddpm_pipeline.to(device)
unet = ddpm_pipeline.unet
unet.eval()

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

UNet2DModel(
  (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=128, out_features=512, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=512, out_features=512, bias=True)
  )
  (down_blocks): ModuleList(
    (0): DownBlock2D(
      (resnets): ModuleList(
        (0-1): 2 x ResnetBlock2D(
          (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
          (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (time_emb_proj): Linear(in_features=512, out_features=128, bias=True)
          (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (nonlinearity): SiLU()
        )
      )
      (downsamplers): ModuleList(
        (0): Downsample2D(
          (conv): Conv2d(12

In [23]:
# Load CIFAR-10 Data

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize images to [-1, 1]
])

# Training data
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# Test data
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
# Define the indices of the layers you want to extract features from
layer_indices = [0, 1, 2, 3]  # Change these indices based on your model structure

# Initialize the modified DDPM model wrapper
ddpm_with_features = DDPMWithFeatures(ddpm_pipeline).to(device)

# Extract features from different layers of the DDPM model
diffusion_train_features, diffusion_train_labels = extract_features_by_layer(
    train_loader, ddpm_with_features, layer_indices, device
)

diffusion_test_features, diffusion_test_labels = extract_features_by_layer(
    test_loader, ddpm_with_features, layer_indices, device
)



Files already downloaded and verified
Files already downloaded and verified


In [24]:
# Train and evaluate classifiers for each layer (same code as previous)
layer_performances = {}

for layer_index in layer_indices:
    print(f"\nTraining classifier for features from layer {layer_index}...")

    # Get the shape of the features to define input dimensions
    input_dim = diffusion_train_features[layer_index].shape[1]

    # Train the classifier
    classifier = train_classifier(diffusion_train_features[layer_index], diffusion_train_labels, input_dim, device)

    # Evaluate the classifier
    print(f"Evaluating classifier for features from layer {layer_index}...")
    accuracy = evaluate_classifier(classifier, diffusion_test_features[layer_index], diffusion_test_labels, device)

    layer_performances[layer_index] = accuracy
    print(f"Layer {layer_index} Accuracy: {accuracy:.2f}%")

# Determine the best layer based on accuracy
best_layer = max(layer_performances, key=layer_performances.get)
print(f"\nBest Layer for Feature Extraction: {best_layer}, Accuracy: {layer_performances[best_layer]:.2f}%")


Training classifier for features from layer 0...
Epoch [1/10], Loss: 1.7844
Epoch [2/10], Loss: 1.2540
Epoch [3/10], Loss: 1.1568
Epoch [4/10], Loss: 1.0858
Epoch [5/10], Loss: 1.0256
Epoch [6/10], Loss: 0.9932
Epoch [7/10], Loss: 0.9542
Epoch [8/10], Loss: 0.9236
Epoch [9/10], Loss: 0.8797
Epoch [10/10], Loss: 0.8654
Evaluating classifier for features from layer 0...
Layer 0 Accuracy: 66.28%

Training classifier for features from layer 1...
Epoch [1/10], Loss: 1.4707
Epoch [2/10], Loss: 1.0376
Epoch [3/10], Loss: 0.9452
Epoch [4/10], Loss: 0.8739
Epoch [5/10], Loss: 0.8100
Epoch [6/10], Loss: 0.7711
Epoch [7/10], Loss: 0.7556
Epoch [8/10], Loss: 0.6981
Epoch [9/10], Loss: 0.6860
Epoch [10/10], Loss: 0.6618
Evaluating classifier for features from layer 1...
Layer 1 Accuracy: 73.37%

Training classifier for features from layer 2...
Epoch [1/10], Loss: 1.3757
Epoch [2/10], Loss: 0.8625
Epoch [3/10], Loss: 0.7757
Epoch [4/10], Loss: 0.7231
Epoch [5/10], Loss: 0.6877
Epoch [6/10], Loss: 0

In [41]:
import pandas as pd
pd.Series(layer_performances).to_csv('data/layer_performances.csv')

In [25]:
# Pre-trained ResNet model
resnet_model = models.resnet18(pretrained=True)
# Remove the final fully connected layer to get feature representations
resnet_model = nn.Sequential(*list(resnet_model.children())[:-1])
resnet_model = resnet_model.to(device)
resnet_model.eval()

def extract_cnn_features(data_loader, model):
    features = []
    labels = []
    with torch.no_grad():
        for images, lbls in data_loader:
            images = images.to(device)
            # Extract features from ResNet
            outputs = model(images).view(images.size(0), -1)  # Flatten output from ResNet
            features.append(outputs.cpu())
            labels.append(lbls)
    features = torch.cat(features)
    labels = torch.cat(labels)
    return features, labels

# Extracting training and testing features
cnn_train_features, cnn_train_labels = extract_cnn_features(train_loader, resnet_model)
cnn_test_features, cnn_test_labels = extract_cnn_features(test_loader, resnet_model)

C:\Users\AliHa\anaconda3\envs\cs764\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\AliHa\anaconda3\envs\cs764\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [39]:
model_scores = {}

In [47]:
cnn_input_dim = cnn_train_features.shape[1]
cnn_classifier = train_classifier(cnn_train_features, cnn_train_labels, cnn_input_dim,device,dropout=0.5)
print("CNN-based Classifier Performance:")
cnn_accuracy = evaluate_classifier(cnn_classifier, cnn_test_features, cnn_test_labels, device)
print(cnn_accuracy)
model_scores['ResNet'] =  [cnn_accuracy, cnn_input_dim]

Epoch [1/10], Loss: 1.1249
Epoch [2/10], Loss: 0.9592
Epoch [3/10], Loss: 0.8992
Epoch [4/10], Loss: 0.8588
Epoch [5/10], Loss: 0.8254
Epoch [6/10], Loss: 0.7906
Epoch [7/10], Loss: 0.7603
Epoch [8/10], Loss: 0.7354
Epoch [9/10], Loss: 0.7061
Epoch [10/10], Loss: 0.6861
CNN-based Classifier Performance:
70.89


# Densenet Model

In [48]:
cnn_model = models.densenet121(pretrained=True)  # or models.densenet169(pretrained=True)
cnn_model = nn.Sequential(*list(cnn_model.children())[:-1])  # Remove the classifier
cnn_model = cnn_model.to(device)
cnn_model.eval()

cnn_train_features, cnn_train_labels = extract_cnn_features(train_loader, cnn_model)
cnn_test_features, cnn_test_labels = extract_cnn_features(test_loader, cnn_model)

In [49]:
cnn_input_dim = cnn_train_features.shape[1]
cnn_classifier = train_classifier(cnn_train_features, cnn_train_labels, cnn_input_dim,device,dropout=0.5)
print("Densenet-based Classifier Performance:")
cnn_accuracy = evaluate_classifier(cnn_classifier, cnn_test_features, cnn_test_labels, device)
print(cnn_accuracy)
model_scores['densenet121'] = [cnn_accuracy, cnn_input_dim]

Epoch [1/10], Loss: 1.1272
Epoch [2/10], Loss: 0.9631
Epoch [3/10], Loss: 0.9049
Epoch [4/10], Loss: 0.8604
Epoch [5/10], Loss: 0.8207
Epoch [6/10], Loss: 0.7902
Epoch [7/10], Loss: 0.7625
Epoch [8/10], Loss: 0.7387
Epoch [9/10], Loss: 0.7043
Epoch [10/10], Loss: 0.6830
Densenet-based Classifier Performance:
70.55


# Efficient Net

In [53]:
cnn_model = models.efficientnet_b0(pretrained=True)
cnn_model = nn.Sequential(*list(cnn_model.children())[:-1])
cnn_model = cnn_model.to(device)
cnn_model.eval()

cnn_train_features, cnn_train_labels = extract_cnn_features(train_loader, cnn_model)
cnn_test_features, cnn_test_labels = extract_cnn_features(test_loader, cnn_model)

C:\Users\AliHa\anaconda3\envs\cs764\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\AliHa/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 56.5MB/s]


In [54]:
cnn_input_dim = cnn_train_features.shape[1]
cnn_classifier = train_classifier(cnn_train_features, cnn_train_labels, cnn_input_dim,device,dropout=0.5)
print("efficientnet_b0-based Classifier Performance:")
cnn_accuracy = evaluate_classifier(cnn_classifier, cnn_test_features, cnn_test_labels, device)
print(cnn_accuracy)
model_scores['efficientnet_b0'] =  [cnn_accuracy, cnn_input_dim]

Epoch [1/10], Loss: 1.5708
Epoch [2/10], Loss: 1.3918
Epoch [3/10], Loss: 1.3418
Epoch [4/10], Loss: 1.3090
Epoch [5/10], Loss: 1.2873
Epoch [6/10], Loss: 1.2716
Epoch [7/10], Loss: 1.2551
Epoch [8/10], Loss: 1.2447
Epoch [9/10], Loss: 1.2286
Epoch [10/10], Loss: 1.2217
efficientnet_b0-based Classifier Performance:
57.6


# MobileNet v2

In [57]:
cnn_model = models.mobilenet_v2(pretrained=True) 
cnn_model = nn.Sequential(*list(cnn_model.children())[:-1])
cnn_model = cnn_model.to(device)
cnn_model.eval()

cnn_train_features, cnn_train_labels = extract_cnn_features(train_loader, cnn_model)
cnn_test_features, cnn_test_labels = extract_cnn_features(test_loader, cnn_model)

C:\Users\AliHa\anaconda3\envs\cs764\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\AliHa/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 48.0MB/s]


In [58]:
cnn_input_dim = cnn_train_features.shape[1]
cnn_classifier = train_classifier(cnn_train_features, cnn_train_labels, cnn_input_dim,device,dropout=0.5)
print("mobilenet_v2-based Classifier Performance:")
cnn_accuracy = evaluate_classifier(cnn_classifier, cnn_test_features, cnn_test_labels, device)
print(cnn_accuracy)
model_scores['mobilenet_v2'] =  [cnn_accuracy, cnn_input_dim]

Epoch [1/10], Loss: 1.5362
Epoch [2/10], Loss: 1.3149
Epoch [3/10], Loss: 1.2421
Epoch [4/10], Loss: 1.1884
Epoch [5/10], Loss: 1.1467
Epoch [6/10], Loss: 1.1057
Epoch [7/10], Loss: 1.0724
Epoch [8/10], Loss: 1.0353
Epoch [9/10], Loss: 0.9994
Epoch [10/10], Loss: 0.9631
mobilenet_v2-based Classifier Performance:
58.25


# ResNeXt-50

In [61]:
cnn_model = models.resnext50_32x4d(pretrained=True)
cnn_model = nn.Sequential(*list(cnn_model.children())[:-1])  
cnn_model = cnn_model.to(device)
cnn_model.eval()

cnn_train_features, cnn_train_labels = extract_cnn_features(train_loader, cnn_model)
cnn_test_features, cnn_test_labels = extract_cnn_features(test_loader, cnn_model)

C:\Users\AliHa\anaconda3\envs\cs764\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to C:\Users\AliHa/.cache\torch\hub\checkpoints\resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:18<00:00, 5.34MB/s]


In [62]:
cnn_input_dim = cnn_train_features.shape[1]
cnn_classifier = train_classifier(cnn_train_features, cnn_train_labels, cnn_input_dim,device,dropout=0.5)
print("resnext50-based Classifier Performance:")
cnn_accuracy = evaluate_classifier(cnn_classifier, cnn_test_features, cnn_test_labels, device)
print(cnn_accuracy)
model_scores['resnext50'] =  [cnn_accuracy, cnn_input_dim]

Epoch [1/10], Loss: 1.1307
Epoch [2/10], Loss: 0.9368
Epoch [3/10], Loss: 0.8800
Epoch [4/10], Loss: 0.8372
Epoch [5/10], Loss: 0.7988
Epoch [6/10], Loss: 0.7696
Epoch [7/10], Loss: 0.7422
Epoch [8/10], Loss: 0.7091
Epoch [9/10], Loss: 0.6880
Epoch [10/10], Loss: 0.6642
resnext50-based Classifier Performance:
71.49


# VGG-16

In [66]:
cnn_model = models.vgg16(pretrained=True)
cnn_model = nn.Sequential(*list(cnn_model.children())[:-1])
cnn_model = cnn_model.to(device)
cnn_model.eval()

cnn_train_features, cnn_train_labels = extract_cnn_features(train_loader, cnn_model)
cnn_test_features, cnn_test_labels = extract_cnn_features(test_loader, cnn_model)

C:\Users\AliHa\anaconda3\envs\cs764\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [67]:
cnn_input_dim = cnn_train_features.shape[1]
cnn_classifier = train_classifier(cnn_train_features, cnn_train_labels, cnn_input_dim,device,dropout=0.5)
print("vgg16-based Classifier Performance:")
cnn_accuracy = evaluate_classifier(cnn_classifier, cnn_test_features, cnn_test_labels, device)
print(cnn_accuracy)
model_scores['vgg-16'] =  [cnn_accuracy, cnn_input_dim]

Epoch [1/10], Loss: 1.3315
Epoch [2/10], Loss: 1.1733
Epoch [3/10], Loss: 1.1358
Epoch [4/10], Loss: 1.1098
Epoch [5/10], Loss: 1.0897
Epoch [6/10], Loss: 1.0722
Epoch [7/10], Loss: 1.0546
Epoch [8/10], Loss: 1.0415
Epoch [9/10], Loss: 1.0288
Epoch [10/10], Loss: 1.0166
vgg16-based Classifier Performance:
65.34


In [77]:
import pandas as pd
pd.DataFrame(model_scores).to_csv('data/model_scores.csv')